# Handling Excel files

Excel files are a very common method for storing and manipulating data. Python has an extensive support for Excel files through the openpyxl module.
This module allows creation of worksheets along with storage, manipulation and reading of data.

## Creating Excel files
The openpyxl module allows the creation of Excel files, manipulation of workbooks and saving the data.

In [8]:
# install openpyxl module if not already
! pip --quiet install openpyxl

# import the openpyxl module
import openpyxl as excel


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# creating excel workbooks and worksheets
work_book = excel.Workbook()

# remove the active sheet
work_book.remove(work_book.active)

# create additional data sheets
work_book.create_sheet("Initial Information")
work_book.create_sheet("Calculated Data")

# finally the workbook can be saved and closed
work_book.save("data.xlsx") 
work_book.close()

## Adding data to Excel files
It is also possible to write data in workbooks at various columns and cells.

In [10]:
# let's reopen the saved notebook
work_book = excel.open("data.xlsx")

# let's display the existing data sheets
print("The sheets in workbook are: {0}".format(work_book.sheetnames))

The sheets in workbook are: ['Initial Information', 'Calculated Data']


In [11]:
import datetime
import random

# let's generate some data and write it into the worksheet
# we will simulate a failure log for a robot and write it into excel
FAILURE_MODE = ["Loader", "Engine", "Control"]
START_FAILURE_DATE = cdt_custom = datetime.datetime.strptime("01-01-2023", "%d-%m-%Y")
failure_data = []

for index in range(0,100):
        
    failure_date = None
    if (index == 0):
        failure_date = START_FAILURE_DATE
    else: 
        failure_date = failure_data[-1]["failure_end_date"]

    failure_end_date = failure_date + datetime.timedelta(random.randint(2,10))
    
    failure_mode = FAILURE_MODE[random.randint(0,2)]
    
    failure_record = {
        "failure_date": failure_date,
        "failure_end_date": failure_end_date,
        "failure_mode": failure_mode
    }
    
    failure_data.append(failure_record)        

In [12]:
# it is possible to access a sheet by its name
work_sheet = work_book["Initial Information"]

# write data header
work_sheet.cell(1, 1, "Failure Date").font = excel.styles.Font(bold = True)
work_sheet.cell(1, 2, "Failure End Date").font = excel.styles.Font(bold = True)
work_sheet.cell(1, 3, "Failure Mode").font = excel.styles.Font(bold = True)

# let's write the data at each cell
for i in range(0, len(failure_data)) :
    work_sheet.cell(i + 2, 1, failure_data[i]["failure_date"])
    work_sheet.cell(i + 2, 2, failure_data[i]["failure_end_date"])
    work_sheet.cell(i + 2, 3, failure_data[i]["failure_mode"])

# save and close workbook
work_book.save("data.xlsx") 
work_book.close()

In [13]:
# let's reopen the workbook and change data
# we will store days until repair
work_book = excel.open("data.xlsx")

# select the appropriate worksheet
initial_information_sheet = work_book["Initial Information"]
initial_information_sheet.cell(1 , 4, "Days Until Repair").font = excel.styles.Font(bold = True)
current_index = 0

while(True):
    # read appropriate cells
    failure_date_cell = initial_information_sheet.cell(current_index + 2, 1)
    failure_end_date_cell = initial_information_sheet.cell(current_index + 2, 2)
    
    if not isinstance(failure_date_cell.value, datetime.datetime):
        break
    
    # calculate new date
    days_until_repair = (failure_end_date_cell.value - failure_date_cell.value).days
    initial_information_sheet.cell(current_index + 2, 4, days_until_repair)
    
    current_index = current_index + 1 

# force worksheet autofit and best fit
for i in range(1, 5):
    column_letter = excel.utils.get_column_letter(i)
    initial_information_sheet.column_dimensions[column_letter].bestFit = True
    initial_information_sheet.column_dimensions[column_letter].auto_size = True
    

# save and close workbook
work_book.save("data.xlsx") 
work_book.close()

In [14]:
# let's reopen the workbook and use formulas
# for calculating min, max and average days until repair
work_book = excel.open("data.xlsx")
calculated_data_sheet = work_book["Calculated Data"]

# write data labels
calculated_data_sheet.cell(1, 1, "Minimum Days Until Repair").font = excel.styles.Font(bold = True)
calculated_data_sheet.cell(2, 1, "Average Days Until Repair").font = excel.styles.Font(bold = True)
calculated_data_sheet.cell(3, 1, "Maximum Days Until Repair").font = excel.styles.Font(bold = True)

calculated_data_sheet.cell(1, 2, "=MIN('Initial Information'!D2:D101)")
calculated_data_sheet.cell(2, 2, "=AVERAGE('Initial Information'!D2:D101)")
calculated_data_sheet.cell(3, 2, "=MAX('Initial Information'!D2:D101)")

for i in range(1, 2):
    column_letter = excel.utils.get_column_letter(i)
    calculated_data_sheet.column_dimensions[column_letter].bestFit = True
    calculated_data_sheet.column_dimensions[column_letter].auto_size = True

# save and close workbook
work_book.save("data.xlsx") 
work_book.close()